In [2]:
import yaml
import os
import numpy as np
import glob
from matplotlib import pyplot as plt
from pycorr import TwoPointCorrelationFunction, TwoPointEstimator, NaturalTwoPointEstimator, utils, setup_logging
setup_logging()
import pandas as pd
import seaborn as sns
sns.set_theme(style="white")

[000000.34]  05-10 20:39  numexpr.utils                INFO     Note: detected 256 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[000000.34]  05-10 20:39  numexpr.utils                INFO     Note: NumExpr detected 256 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


# Prepare Y1 measurements

In [13]:
def readwp(path):
    allcounts=TwoPointEstimator.load(path)
    sep, wp, cov = allcounts[::2].get_corr(mode='wp',return_sep=True)
    err=np.sqrt(np.diag(cov))
    return wp, cov, err, allcounts

In [27]:
rpbins=np.geomspace(0.01,100,25)
rpbinsmid=(rpbins[1:]+rpbins[:-1])/2
rppidir='/pscratch/sd/h/hanyuz/Y1HOD/v1.4pip/rppi/' 
catdir='/global/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.4pip/'
measurementdir='/global/homes/h/hanyuz/AbacusHODGuide/datas/' #update path
#measurement and covmat root name
wpprefix = 'wp_LRG_GCcomb'
covprefix = 'jkcov_LRG_GCcomb'

## read measurements

In [15]:
l0406=readwp(rppidir+'allcounts_LRG_GCcomb_0.4_0.6_pip_angular_bitwise_log_njack128_nran8_split20.npy')
l0608=readwp(rppidir+'allcounts_LRG_GCcomb_0.6_0.8_pip_angular_bitwise_log_njack128_nran8_split20.npy')
l0811=readwp(rppidir+'allcounts_LRG_GCcomb_0.8_1.1_pip_angular_bitwise_log_njack128_nran8_split20.npy')

[000667.79]  05-10 20:50  TwoPointEstimator            INFO     Loading /pscratch/sd/h/hanyuz/Y1HOD/v1.4pip/rppi/allcounts_LRG_GCcomb_0.4_0.6_pip_angular_bitwise_log_njack128_nran8_split20.npy.
[000668.67]  05-10 20:50  TwoPointEstimator            INFO     Loading /pscratch/sd/h/hanyuz/Y1HOD/v1.4pip/rppi/allcounts_LRG_GCcomb_0.6_0.8_pip_angular_bitwise_log_njack128_nran8_split20.npy.
[000669.43]  05-10 20:50  TwoPointEstimator            INFO     Loading /pscratch/sd/h/hanyuz/Y1HOD/v1.4pip/rppi/allcounts_LRG_GCcomb_0.8_1.1_pip_angular_bitwise_log_njack128_nran8_split20.npy.


## save to datas folder

wp measurements, columns: rpmid, wp, jkerr

In [28]:
np.savetxt(measurementdir+wpprefix+'_0.4_0.6_pip_angular_bitwise_v1.4pip.dat',np.column_stack((rpbinsmid,l0406[0],l0406[2])))
np.savetxt(measurementdir+wpprefix+'_0.6_0.8_pip_angular_bitwise_v1.4pip.dat',np.column_stack((rpbinsmid,l0608[0],l0608[2])))
np.savetxt(measurementdir+wpprefix+'_0.8_1.1_pip_angular_bitwise_v1.4pip.dat',np.column_stack((rpbinsmid,l0811[0],l0811[2])))

jkcovmat, better cut to the scale we want to fit, for LRG 3:21, for QSO 7:21 (only cut covmat, do not cut wp)

In [29]:
cutmask = np.ix_(np.arange(3, 21), np.arange(3, 21))
np.savetxt(measurementdir+covprefix+'_0.4_0.6_pip_angular_bitwise_v1.4pip.dat',l0406[1][cutmask])
np.savetxt(measurementdir+covprefix+'_0.6_0.8_pip_angular_bitwise_v1.4pip.dat',l0608[1][cutmask])
np.savetxt(measurementdir+covprefix+'_0.8_1.1_pip_angular_bitwise_v1.4pip.dat',l0811[1][cutmask])

## n(z)

In [79]:
def readnz(root,skip,maxrows):
    ngc=np.loadtxt(root+'_NGC_nz.txt',skiprows=skip,max_rows=maxrows)
    sgc=np.loadtxt(root+'_SGC_nz.txt',skiprows=skip,max_rows=maxrows)  
    gccomb=(np.sum(ngc[:,-2])+np.sum(sgc[:,-2]))/(np.sum(ngc[:,-1])+np.sum(sgc[:,-1]))
    return gccomb,0.1*gccomb

In [80]:
nzroot=catdir+'LRG'
nz_l0406,std_l0406=readnz(nzroot,2,20)
nz_l0608,std_l0608=readnz(nzroot,22,20)
nz_l0811,std_l0811=readnz(nzroot,42,30)

# Prepare configuration files for AbacusHOD

## path configs

In [30]:
#path to subsamples
subsampledir = '/pscratch/sd/h/hanyuz/Y1HOD/subsample_base_quadown/'
#path to populate mocks
mockoutdir = '/pscratch/sd/h/hanyuz/Y1HOD/mocks/'
#path to measurements and covmat for fitting
measurementdir = '/global/homes/h/hanyuz/AbacusHODGuide/datas/'
#path to save chains and checkpoints
chainoutdir = '/pscratch/sd/h/hanyuz/Y1HOD/chains/'
#chains root name
chainsprefix = 'chain_LRG_GCcomb'
#want velocity bias? set to True only if you are fitting to wp+xi0+xi2
want_vb = False

## LRG configs

In [73]:
#redshiftbins of data
zbin = [[0.4, 0.6],[0.6, 0.8],[0.8, 1.1]]
#redshift of simulation that used to fit the data
zsnap = [0.5, 0.8, 1.1]
#number density of corresponding redshiftbins
nz = [nz_l0406,nz_l0608,nz_l0811]
#take 10% as number density error
stdnz=[std_l0406,std_l0608,std_l0811]
#Initial values for velocity parameters
#since we are fitting only wp, we could fix velocity bias parameters to SecondGen value
#If we are fitting to wp+xi0+xi2, we dont care about the initial values since they could be free params
alpha_c = [0.33, 0.19, 0.42] 
alpha_s = [0.80, 0.95, 0.92]

In [82]:
template_config_path='configs/template_run_LRG.yaml'
with open(template_config_path, 'r') as file:
    template_config = yaml.safe_load(file)
for zidx in range(3):
    config_name = f'run_LRG_GCcomb_{zbin[zidx][0]}_{zbin[zidx][1]}_pip_angular_bitwise_v1.4pip_zheng07'
    template_config['HOD_params']['LRG_params']['alpha_c'] = alpha_c[zidx]
    template_config['HOD_params']['LRG_params']['alpha_s'] = alpha_s[zidx]
    template_config['sim_params']['z_mock'] = zsnap[zidx]
    template_config['sim_params']['output_dir'] = mockoutdir
    template_config['data_params']['tracer_density_mean']['LRG'] = float(nz[zidx])
    template_config['data_params']['tracer_density_std']['LRG'] = float(stdnz[zidx])
    template_config['data_params']['tracer_combos']['LRG_LRG']['path2wp'] = measurementdir+wpprefix+f'_{zbin[zidx][0]}_{zbin[zidx][1]}_pip_angular_bitwise_v1.4pip.dat'  
    template_config['fit_config_params']['chainsPrefix'] = chainsprefix+f'_{zbin[zidx][0]}_{zbin[zidx][1]}_pip_angular_bitwise_v1.4pip'
    template_config['fit_config_params']['joint'] = measurementdir+covprefix+f'_{zbin[zidx][0]}_{zbin[zidx][1]}_pip_angular_bitwise_v1.4pip.dat'
    template_config['fit_config_params']['path2output'] = chainoutdir
    config_path = f'/global/homes/h/hanyuz/AbacusHODGuide/configs/{config_name}.yaml'#change path
    if want_vb:
        template_config['fit_config_params']['LRG']['alpha_c'] = [5,0,1,0.4,0.4]
        template_config['fit_config_params']['LRG']['alpha_s'] = [6,0,2,0.8,0.4]
        config_name+='_velbias'
        
    with open(config_path, 'w') as new_file:
        yaml.safe_dump(template_config, new_file, default_flow_style=False)

    print(f'Config file for {config_name} has been saved to {config_path}')  

Config file for run_LRG_GCcomb_0.4_0.6_pip_angular_bitwise_v1.4pip_zheng07 has been saved to /global/homes/h/hanyuz/AbacusHODGuide/configs/run_LRG_GCcomb_0.4_0.6_pip_angular_bitwise_v1.4pip_zheng07.yaml
Config file for run_LRG_GCcomb_0.6_0.8_pip_angular_bitwise_v1.4pip_zheng07 has been saved to /global/homes/h/hanyuz/AbacusHODGuide/configs/run_LRG_GCcomb_0.6_0.8_pip_angular_bitwise_v1.4pip_zheng07.yaml
Config file for run_LRG_GCcomb_0.8_1.1_pip_angular_bitwise_v1.4pip_zheng07 has been saved to /global/homes/h/hanyuz/AbacusHODGuide/configs/run_LRG_GCcomb_0.8_1.1_pip_angular_bitwise_v1.4pip_zheng07.yaml
